In [ ]:
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from geopy.distance import geodesic
import lightgbm as lgb

In [ ]:
data = pd.read_csv("C:/Users/spkr1/Downloads/CREDIT CARD FRAUD DETECTION/dataset.csv")
df = data.copy()
df[df['is_fraud']==1].head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
2449,2449,2019-01-02 01:06:37,4613314721966,fraud_Rutherford-Mertz,grocery_pos,281.06,Jason,Murphy,M,542 Steve Curve Suite 011,...,35.9946,-81.7266,885,Soil scientist,1988-09-15,e8a81877ae9a0a7f883e15cb39dc4022,1325466397,36.430124,-81.179483,1
2472,2472,2019-01-02 01:47:29,340187018810220,"fraud_Jenkins, Hauck and Friesen",gas_transport,11.52,Misty,Hart,F,27954 Hall Mill Suite 575,...,29.4400,-98.4590,1595797,Horticultural consultant,1960-10-28,bc7d41c41103877b03232f03f1f8d3f5,1325468849,29.819364,-99.142791,1
2523,2523,2019-01-02 03:05:23,340187018810220,fraud_Goodwin-Nitzsche,grocery_pos,276.31,Misty,Hart,F,27954 Hall Mill Suite 575,...,29.4400,-98.4590,1595797,Horticultural consultant,1960-10-28,b98f12f4168391b2203238813df5aa8c,1325473523,29.273085,-98.836360,1
2546,2546,2019-01-02 03:38:03,4613314721966,fraud_Erdman-Kertzmann,gas_transport,7.03,Jason,Murphy,M,542 Steve Curve Suite 011,...,35.9946,-81.7266,885,Soil scientist,1988-09-15,397894a5c4c02e3c61c784001f0f14e4,1325475483,35.909292,-82.091010,1
2553,2553,2019-01-02 03:55:47,340187018810220,fraud_Koepp-Parker,grocery_pos,275.73,Misty,Hart,F,27954 Hall Mill Suite 575,...,29.4400,-98.4590,1595797,Horticultural consultant,1960-10-28,7863235a750d73a244c07f1fb7f0185a,1325476547,29.786426,-98.683410,1


In [19]:
data["trans_date_trans_time"] = pd.to_datetime(data["trans_date_trans_time"])

data["hour"] = data["trans_date_trans_time"].dt.hour
data["day"] = data["trans_date_trans_time"].dt.day
data["month"] = data["trans_date_trans_time"].dt.month


In [20]:
from geopy.distance import geodesic

def calculate_distance(row):
    return geodesic(
        (row["lat"], row["long"]),
        (row["merch_lat"], row["merch_long"])
    ).km

data["distance"] = data.apply(calculate_distance, axis=1)

In [21]:
encoders = {}
categorical_cols = ["merchant", "category", "gender"]

for col in categorical_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    encoders[col] = le


In [22]:
data["cc_num"] = data["cc_num"].astype(str).apply(
    lambda x: hash(x) % 100
)

X = data[[
    "merchant", "category", "amt", "distance",
    "hour", "day", "month", "gender", "cc_num"
]]

y = data["is_fraud"]   # or "Class"


In [23]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

model = lgb.LGBMClassifier(
    n_estimators=100,
    learning_rate=0.1,
    random_state=42
)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


[LightGBM] [Info] Number of positive: 6005, number of negative: 1031335
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009736 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 948
[LightGBM] [Info] Number of data points in the train set: 1037340, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.005789 -> initscore=-5.146017
[LightGBM] [Info] Start training from score -5.146017
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    257834
           1       0.73      0.68      0.70      1501

    accuracy                           1.00    259335
   macro avg       0.87      0.84      0.85    259335
weighted avg       1.00      1.00      1.00    259335



In [24]:
joblib.dump(model, "fraud_detection_model.jb")
joblib.dump(encoders, "label_encoder.jb")

print("✅ Model and encoders saved successfully")


✅ Model and encoders saved successfully
